### Implementing text generation with Keras

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import text_dataset_from_directory
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras import layers
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.layers import Embedding, TextVectorization, Dropout, Dense, Layer
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Embedding
from tensorflow.keras.callbacks import Callback

**Dataset Preparation**

In [2]:
def clean_data(text):

    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text, r'[^\x00-\x7F]+', '')  
    text = tf.strings.regex_replace(text, r"[^a-zA-Z0-9\s.,!?']", "")
    text = tf.strings.regex_replace(text, "<br />", " ")
    
    return text

In [3]:
data = text_dataset_from_directory(directory="aclImdb", label_mode=None, batch_size=256)

Found 100006 files belonging to 1 classes.


In [4]:
dataset = data.map(clean_data)

In [5]:
sequence_length = 100
vocab_size = 15000

In [6]:
vectorization = TextVectorization(max_tokens=vocab_size, output_mode="int", output_sequence_length=sequence_length)
vectorization.adapt(dataset)

**Setting up a language modeling dataset**

In [7]:
def prepare_dataset(text_batch):
    
    vectorized_sequences = vectorization(text_batch)
    x = vectorized_sequences[:, :-1]
    y = vectorized_sequences[:, 1:]
    
    return x, y

In [8]:
lm_dataset = dataset.map(prepare_dataset, num_parallel_calls=4)

#### Positional Embedding

In [9]:
class PositionalEmbedding(Layer):
    
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        
        super().__init__(**kwargs)
        self.token_embeddings = Embedding(input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = Embedding(input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        
        return config

#### Decoder

In [10]:
class TransformerDecoder(Layer):
    
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = Sequential([Dense(dense_dim, activation="relu"), Dense(embed_dim)])
        self.layernorm_1 = LayerNormalization()
        self.layernorm_2 = LayerNormalization()
        self.layernorm_3 = LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        
        config = super(TransformerDecoder, self).get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        
        return config

    def get_causal_attention_mask(self, inputs):
        
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat([tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], axis=0)
        
        return tf.tile(mask, mult)

    def call(self, inputs, outputs, mask=None):
        
        causal_mask = self.get_causal_attention_mask(inputs)
        
        if mask is not None:
            attention_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            attention_mask = tf.minimum(attention_mask, causal_mask)
        else:
            attention_mask = mask
            
        attention_output_1 = self.attention_1(query=inputs, value=inputs, key=inputs, attention_mask=causal_mask)
        query = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(query=query, value=outputs, key=outputs, attention_mask=attention_mask)
        attention_output_2 = self.layernorm_2(attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        
        return self.layernorm_3(attention_output_2 + proj_output)

**A simple Transformer-based language model**

In [11]:
embed_dim = 256
latent_dim = 2048
num_heads = 2

In [12]:
inputs = Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, x)
outputs = Dense(vocab_size, activation="softmax")(x)
model = Model(inputs, outputs)
model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop")

**The text-generation callback**

In [13]:
tokens_index = dict(enumerate(vectorization.get_vocabulary()))

def sample_next(predictions, temperature=1.0):
    
    predictions = np.asarray(predictions).astype("float64")
    predictions = np.log(predictions) / temperature
    exp_preds = np.exp(predictions)
    predictions = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, predictions, 1)
    
    return np.argmax(probas)

class TextGenerator(Callback):
    
    def __init__(self, prompt, generate_length, model_input_length, temperatures=(1.,), print_freq=1):
        
        self.prompt = prompt
        self.generate_length = generate_length
        self.model_input_length = model_input_length
        self.temperatures = temperatures
        self.print_freq = print_freq
        vectorized_prompt = vectorization([prompt])[0].numpy()
        self.prompt_length = np.nonzero(vectorized_prompt == 0)[0][0]

    def on_epoch_end(self, epoch, logs=None):
        
        if (epoch + 1) % self.print_freq != 0:
            return
        
        for temperature in self.temperatures:
            
            print("== Generating with temperature", temperature)
            sentence = self.prompt
            
            for i in range(self.generate_length):
                tokenized_sentence = vectorization([sentence])
                predictions = self.model(tokenized_sentence)
                next_token = sample_next(predictions[0, self.prompt_length - 1 + i, :])
                sampled_token = tokens_index[next_token]
                sentence += " " + sampled_token
                
            print(sentence)

In [14]:
prompt = "This movie"
text_generator = TextGenerator(prompt, generate_length=50, model_input_length=sequence_length, temperatures=(0.5, 0.7, 0.8))

**Fitting the language model**

In [15]:
model.fit(lm_dataset, epochs=200, callbacks=[text_generator])

Epoch 1/200
391/391 [==============================] - ETA: 0s - loss: 5.7698== Generating with temperature 0.5
This movie thinks of all movies but why make on the size and pg13 movies a dead clichs i have made at least directed carol tries box larger comedy psychological film quite sad then it sends over ginger downey boy against nowhere then was lacking kidsbr i probably never seen i love
== Generating with temperature 0.7
This movie is william angeles house turkish title film is given [UNK] by his lynchs [UNK] lens and very interesting enough does have been the action horror and busey [UNK] insignificant series that his partner i goes away about the [UNK] go feel to travels to seek up up from start but
== Generating with temperature 0.8
This movie runs off in japan [UNK] that stands [UNK] [UNK] with top but we are because badly made moments watch most performances from a white look rather than the perfect tale of locks in this is water and john chuck [UNK] obviously clearly lin the 

This movie is based on screen has been [UNK] in 2006 the year [UNK] it is from the start counting the river now and so far can this day is as good as the [UNK] are they have this tiny budget as for the [UNK] video games not only make if the
391/391 [==============================] - 83s 211ms/step - loss: 4.3384
Epoch 9/200
391/391 [==============================] - ETA: 0s - loss: 4.2902== Generating with temperature 0.5
This movie seemed to be written edited into an angry cheesy goodfellas and for that its original to see robin needs the owner of a [UNK] his photography gets [UNK] smart conviction and drawn in by the fbi agent everything about the secret council not that blowing up this i am sure
== Generating with temperature 0.7
This movie is boasts some real action sequences to match the other asian [UNK] make one feel that blue falls to mind granted the pro demands of shooting at the lens the acting is bad the incoherent honest [UNK] wasnt even good performances by the brando who

This movie sounded more like [UNK] ending than it could be that is [UNK] great from the original concept would have been more pretty good but the turns of van [UNK] dog and a real delight with zach tin will destroy those [UNK] fright [UNK] her hubby who gets pursued by the
== Generating with temperature 0.8
This movie made no sense whatsoever it unbelievably lame burton tries to make you want to put a bunch of bad effects and bad acting which are so bad it aint funny and i suspect that theres a monster type so see [UNK] sleeping in the middle of the thing that has
391/391 [==============================] - 83s 211ms/step - loss: 4.0792
Epoch 17/200
391/391 [==============================] - ETA: 0s - loss: 4.0592== Generating with temperature 0.5
This movie is a perfect example of what is coming and recommended primarily with im a huge fan of music and the story is a little more than original i expected it to be considered the only way i could drive a [UNK] but i told those artists cou

Epoch 24/200
391/391 [==============================] - ETA: 0s - loss: 3.9558== Generating with temperature 0.5
This movie was at least a year ago when i saw the ads for your standards always being turned me into one of those movies not even any great ones in that it didnt really hold my attention was suppose to address that palette of brain climate ideas were excellent and therefore
== Generating with temperature 0.7
This movie is stuffed soft core sex and beautifully done from most innocent movies in the years tales could have been a better movie if one of her friends dance local chinese [UNK] its not really about dancing in bed with [UNK] david [UNK] everyone comes to think of the good featuring
== Generating with temperature 0.8
This movie was an absolute travesty of a filipino film the painful one has seen more hard times before it probably stands as one of the most talented cartoons i ever sat on the dvd and only viewing i can agree to conquest the movie [UNK] that i have watche

Epoch 32/200
391/391 [==============================] - ETA: 0s - loss: 3.8828== Generating with temperature 0.5
This movie is so bad its funny this movie was hilarious and it call it god its that trick you can really call it a comedy to make you laugh stolen by the typical actorsactresses from a wide variety of reasons the sinister humorbr br the plot is totally silly and the
== Generating with temperature 0.7
This movie fit what is you and not in hell to remember you saw most of this movie you know the start it i think i enjoyed watching the horrible dialog but rushed i thought the acting was surprisingly good as well as the vocals were very predictable this movie is pretty
== Generating with temperature 0.8
This movie was ok it was as bad as some sequels go this review okay i tried i mean hold my attention to its awesome its nice to make up for the sequel but 12 for the scene where spectacular movie was pretty bad but then i sat down and watched then
391/391 [=======================

...

This movie simply sucked in incredibly bad taste make up some lame fights that keep me entertained throughout the entire moviebr br quite obvious to me suck person 2 crappy girls i think that green lit was on a cheap budget 2 special effects taken a future royal restaurant just like i
391/391 [==============================] - 84s 213ms/step - loss: 3.6121
Epoch 163/200
391/391 [==============================] - ETA: 0s - loss: 3.6114== Generating with temperature 0.5
This movie is a far from its annoying considering its very familiar viewing with the low quality of its script and the jokes are very weak it lacks parody of the jokes are unclear the scenes are so very few the quality the grace the music david [UNK] does this much like
== Generating with temperature 0.7
This movie is a torture chamber is unforgettable in a manner the dubbing of vampires zombies who keep you at the local [UNK] especially been given a dollar work of the screenplay the dialogue the director and the actor th

This movie consists of terrible characters i am an enormously sympathetic character actor as far as i do not want my life back i could not bring back the same basic plotline was the fact that such [UNK] have a huge success in british tv and film but given its chance and
== Generating with temperature 0.8
This movie was most unsettling and [UNK] the plot was all bad especially was only he could play in one of his high school graduation row musical when he despite the problem for this the south park ranger it was another sign of porn these teens looked [UNK] was already a total
391/391 [==============================] - 86s 219ms/step - loss: 3.6057
Epoch 171/200
391/391 [==============================] - ETA: 0s - loss: 3.6049== Generating with temperature 0.5
This movie was [UNK] january 1st and i remember speak two is quite good goldblum actresses and turn a impression of a clothes full of dialog and a static representation of what the movie was about to make the director [UNK] leroy 

Epoch 178/200
391/391 [==============================] - ETA: 0s - loss: 3.5996== Generating with temperature 0.5
This movie was one of the worst films i have seen i have two things to say about [UNK] and [UNK] view of what is [UNK] that they gave old information in this film and the funniest part is when done the german film is pointless and ta i think it was
== Generating with temperature 0.7
This movie is always considered very good and good i dont see why this movie was made i always have the future should you only watch it if you can get some kind of movie regular and to love and hate the adrenaline people and fed up with the stories and less
== Generating with temperature 0.8
This movie was absolutely disappointmentbr br shot in movie does anyone remember it from the final scene of the movie richard hatch was so bland that they were making a lot more realistic as the other lead but given the first section of this movie lacks focus direction and red riding hood
391/391 [==========

This movie was brilliant it has everything colors and what modern middle class actors and actresses have done well in this case problem is mainly because its slow bonus airplane sports then [UNK] native americans the hooks break pop riots fakes re released of effect some school in terms of gender to
391/391 [==============================] - 85s 216ms/step - loss: 3.5946
Epoch 186/200
391/391 [==============================] - ETA: 0s - loss: 3.5939== Generating with temperature 0.5
This movie has an interesting mood its expected from a treat he hilariously rude and [UNK] billy bob thornton are back for [UNK] to william daniels [UNK] and [UNK] tate donovan begin opening up a little seen and when its aged absolutely not the worst thing ive ever seen the kissing would
== Generating with temperature 0.7
This movie shows that fights stand out hate some [UNK] directed by george c scott who is he served against people who might have born before that time home this is the only part of this mo

This movie is about what i would call a bad movie i wanted after [UNK] decent compared to is the 1980s corey feldman act or hunter robert deniro as james [UNK] this [UNK] is [UNK] is extremely evil to watch at the very worst [UNK] bad guy although he doesnt mean it
== Generating with temperature 0.8
This movie is one of those late 80s slasher films that give you some bad ones like any of your previous and expect a good low budget film i was really looking forward to thinking it would be a some parisian [UNK] with argento everything that ed but also has some decent
391/391 [==============================] - 88s 223ms/step - loss: 3.5893
Epoch 194/200
391/391 [==============================] - ETA: 0s - loss: 3.5888== Generating with temperature 0.5
This movie was never considered to be serious what it was about [UNK] timing a wedding starting with mary the beginning of girls trying to [UNK] what thats like whats coming of her grandfather this adds comical quotes and leaves most pointles